In [1]:
import arcpy
import requests
import tarfile
import requests
import zipfile
import io
import urllib.request
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import os
import fiona
from datetime import datetime

In [2]:
arcpy.env.workspace = r'E:\ArcGIS_1\Project\BP'
wksp = arcpy.env.workspace

# Data acquisition

In [3]:
# Oil plume trajectory based on NOAA's data
oil_link = r'http://maps1.arcgisonline.com/arcgis/rest/services/NOAA_Gulf_Oil_Spill/MapServer/0/query?where=SHAPE_Area%3E0&text=&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelIntersects&relationParam=&outFields=*&returnGeometry=true&returnTrueCurves=false&maxAllowableOffset=&geometryPrecision=&outSR=&returnIdsOnly=false&returnCountOnly=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&returnZ=false&returnM=false&gdbVersion=&returnDistinctValues=false&resultOffset=&resultRecordCount=&queryByDistance=&returnExtentsOnly=false&datumTransformation=&parameterValues=&rangeValues=&f=pjson'
oil_output = requests.get(oil_link)

with open(r'oil.json', 'wb') as data:
    data.write(oil_output.content)
    
arcpy.conversion.JSONToFeatures("oil.json", "oil_spill.shp", "POLYGON")

<Result 'E:\\ArcGIS_1\\Project\\BP\\oil_spill.shp'>

In [4]:
# US borders
border_link = r'https://www2.census.gov/geo/tiger/GENZ2018/shp/cb_2018_us_nation_5m.zip'
border_output = requests.get(border_link)
zipfile.ZipFile(io.BytesIO(border_output.content)).extractall(wksp)

arcpy.management.Project("cb_2018_us_nation_5m.shp", "border_Project.shp", 'GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]', "WGS_1984_(ITRF00)_To_NAD_1983", 'GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]', "NO_PRESERVE_SHAPE", None, "NO_VERTICAL")

<Result 'E:\\ArcGIS_1\\Project\\BP\\border_Project.shp'>

In [5]:
# Water physicochemical variables
water_link = r'https://www.ncei.noaa.gov/archive/archive-management-system/OAS/bin/prd/jquery/download/117436.2.2.tar.gz'

with requests.get(water_link, stream=True) as  rx, tarfile.open(fileobj=rx.raw, mode="r:gz") as tarobj: 
        tarobj.extractall(wksp)

# Data cleaning and preparation

In [6]:
# Folder for clean water data
arcpy.CreateFolder_management(wksp, "water_data")

<Result 'E:\\ArcGIS_1\\Project\\BP\\water_data'>

In [7]:
# Rewrite text files omitting dash-populated lines
def clean_txt(txt):
    with open(txt, 'r') as file:
        lines = file.readlines()
    
    name = arcpy.da.Describe(txt)['baseName'] + '.txt'
    path = os.path.join(wksp, 'water_data', name)
    with open(path, 'w') as file:
        for line in lines:
            if line[0] == '-':
                continue
            else:
                file.write(line)

In [8]:
# Clean files with the station locations and water variables
raw_data = r'0117436\2.2\data\1-data'
directory = os.path.join(wksp, raw_data)
files = ['2010_Seabird', '2010_StationList']
for filename in os.listdir(directory):
    for item in files:
        if item in filename:
            f = os.path.join(directory, filename)
            clean_txt(f)

### Stations

In [9]:
# Create and clean station DataFrame
stations = pd.read_csv(r'water_data\2010_StationList.txt', sep='|')
stations.drop(stations.iloc[:, 7:17], inplace=True, axis=1)
stations.drop(stations.columns[[0, 1]], axis=1, inplace=True)

# Remove empty rows in the table tail
stations.drop(stations.index[237:242], inplace=True) 

# Remove whitespaces from the header
stations.rename(columns = {
    "     StationID      ": "StationID",
    "      Station       ": "Station",
    "        Date        ": "Date",
    "       LAT dd       ": "Latitude",
    "       LON dd       ": "Longitude",
},
inplace=True
)

In [10]:
# Remove whitespaces from cells
for row in range(len(stations)):
    for col in range(len(list(stations.columns))):
        stations.iloc[row, col] = stations.iloc[row, col].strip()

In [11]:
# Remove stations without coordinates
empty_coords = list()
for row in range(len(stations)):
    if stations.iloc[row, 3] == '':
        empty_coords.append(row)
        
stations = stations.drop(empty_coords)

In [12]:
# Create point geometry
stations['geometry'] = stations.apply(lambda x: Point((float(x.Longitude), float(x.Latitude))), axis=1)

# Drop Lat and Long columns, and create GeoDataFrame
stations.drop(stations.columns[[3, 4]], axis=1, inplace=True)
stations_gdf = gpd.GeoDataFrame(stations, geometry = 'geometry')
stations_gdf

,StationID,Station,Date,geometry
0,1,C6C,2/14/2010,POINT (-90.48950 28.86880)
2,2,F0,3/22/2010,POINT (-91.61450 29.27220)
3,3,F1,3/22/2010,POINT (-91.61530 29.18270)
4,4,F2,3/22/2010,POINT (-91.61480 29.05000)
5,5,F2A,3/22/2010,POINT (-91.57630 28.94730)
...,...,...,...,...
232,213,C6C,10/28/2010,POINT (-90.49070 28.86870)
233,213-2,C6C-2,10/28/2010,POINT (-90.49180 28.86550)
234,212,C7,10/28/2010,POINT (-90.39150 28.83180)
235,211,C8,10/28/2010,POINT (-90.27700 28.78870)


### Water variables

In [13]:
# Create and clean water variables DataFrame
seabird = pd.read_csv(r'water_data\2010_Seabird.txt', sep='|')
seabird.drop(seabird.columns[[0, 1, 17]], axis=1, inplace=True)

# Remove whitespaces from the DataFrame
header = []
for i in range(len(seabird.columns.values)):
    header.append(seabird.columns.values[i].strip())
    
seabird.set_axis(header, axis=1, inplace=True)

for row in range(len(seabird)):
    for col in range(len(list(seabird.columns))):
        if type(seabird.iloc[0, col]) == str:
            seabird.iloc[row, col] = seabird.iloc[row, col].strip()

In [14]:
# Add a new column by concatenating Date and StationID values to create a temporary ID
seabird['Date & StationID'] = seabird['Date'].str.cat(seabird['StationID'], sep = " ")
seabird

,StationID,Date,Station,Scan,Depth (m),Alt (m),Irradiance (%),Transmiss (%),Fluorescence (RFU),Temperature (C),Conductivity (S/m),Salinity (PSU),Density (kg/m^3),Oxygen (mg/L),Oxygen (%),Date & StationID
0,17,3/23/2010,C1,832,0.26,4.13,,22.67,0.90,17.15,3.46,25.78,18.42,8.36,100.93,3/23/2010 17
1,17,3/23/2010,C1,902,1.05,3.81,68.13,23.29,0.84,17.10,3.49,26.08,18.66,8.38,101.27,3/23/2010 17
2,17,3/23/2010,C1,912,1.12,3.69,57.72,23.29,0.85,17.10,3.48,26.05,18.64,8.40,101.43,3/23/2010 17
3,17,3/23/2010,C1,952,1.24,3.32,44.52,23.09,0.88,17.09,3.47,25.98,18.59,8.35,100.70,3/23/2010 17
4,17,3/23/2010,C1,960,1.30,3.35,43.77,22.78,0.89,17.08,3.48,25.99,18.60,8.30,100.18,3/23/2010 17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7727,209,10/27/2010,F5,7791,27.83,0.93,1.58,10.98,0.28,26.23,5.53,35.50,23.46,3.99,60.57,10/27/2010 209
7728,209,10/27/2010,F5,7770,27.93,0.88,1.47,9.73,0.31,26.24,5.53,35.50,23.46,4.00,60.71,10/27/2010 209
7729,209,10/27/2010,F5,7807,28.03,0.90,1.37,9.94,0.25,26.23,5.53,35.50,23.46,4.01,60.84,10/27/2010 209
7730,209,10/27/2010,F5,7675,28.05,6.67,1.34,9.37,0.29,26.23,5.53,35.50,23.46,3.96,60.17,10/27/2010 209


In [15]:
# Auxiliary dataframe to find the minimum depth for each station on each date
auxiliary_df = seabird.groupby('Date & StationID').agg({'Depth (m)': 'min'})
auxiliary_df.reset_index(inplace=True)
auxiliary_df['Depth (m)'] = auxiliary_df['Depth (m)'].astype(str)
auxiliary_df

,Date & StationID,Depth (m)
0,10/27/2010 203,0.03
1,10/27/2010 204,0.05
2,10/27/2010 205,0.07
3,10/27/2010 206,0.04
4,10/27/2010 207,0.03
...,...,...
206,9/17/2010 198,0.29
207,9/17/2010 199,0.17
208,9/17/2010 200,0.2
209,9/18/2010 222,0.04


In [16]:
# Create keys for join with unique ID by concatenating data, station ID, and depth 
auxiliary_df['Join key'] = auxiliary_df['Date & StationID'].str.cat(auxiliary_df['Depth (m)'], sep = " ")
seabird['Join key'] = seabird['Date & StationID'].str.cat(seabird['Depth (m)'].astype(str), sep = " ")

In [17]:
# The join output keeps only the readings at the minimum depth for each station on each date
join = auxiliary_df.merge(seabird, left_on='Join key', right_on='Join key', how = 'inner')
join.drop(join.columns[[0, 1, 2, 9, 18]], axis=1, inplace=True)
join.rename(columns = {'Depth (m)_y': 'Depth (m)'}, inplace = True)
join

,StationID,Date,Station,Scan,Depth (m),Alt (m),Transmiss (%),Fluorescence (RFU),Temperature (C),Conductivity (S/m),Salinity (PSU),Density (kg/m^3),Oxygen (mg/L),Oxygen (%)
0,203,10/27/2010,F0,2263,0.03,1.93,0.02,0.67,26.02,3.92,24.51,15.14,5.89,83.76
1,204,10/27/2010,F1,1022,0.05,5.89,45.07,0.24,25.76,4.01,25.29,15.80,6.48,92.08
2,205,10/27/2010,F2,1633,0.07,5.98,27.28,0.13,26.07,4.65,29.47,18.85,6.22,91.02
3,206,10/27/2010,F2A,1549,0.04,13.97,82.12,0.04,26.24,4.91,31.14,20.05,6.58,97.49
4,207,10/27/2010,F3,353,0.03,17.80,86.29,0.02,26.25,4.89,31.04,19.97,6.69,99.06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
207,198,9/17/2010,F2,974,0.29,5.10,71.59,0.34,29.79,4.93,29.11,17.39,6.74,105.18
208,199,9/17/2010,F1,1583,0.17,5.05,35.77,0.30,30.25,4.57,26.46,15.26,6.35,98.49
209,200,9/17/2010,F0,825,0.20,2.05,4.66,0.36,30.09,4.36,25.23,14.39,6.29,96.53
210,222,9/18/2010,F2A-2,1131,0.04,14.21,26.31,0.22,29.66,5.05,29.95,18.06,6.54,102.40


In [18]:
# Add the water variable reading at the minimum depth to the station GeoDataFrame
data = stations_gdf.merge(join, left_on='StationID', right_on='StationID', how = 'inner')
data.drop(data.columns[[1, 2, 6, 8, 9, 10, 11, 13, 14]],  axis=1, inplace=True)
header_data = ['Station ID', 'geometry', 'Date', 'Station', 'Depth (m)', 'Cond (S/m)', 'O2 (mg/L)', 'O2 (% sat)']
data.set_axis(header_data, axis=1, inplace=True)

# Drop readings before the oil spill accident
data.drop(data.index[0:34], inplace=True)
data

,Station ID,geometry,Date,Station,Depth (m),Cond (S/m),O2 (mg/L),O2 (% sat)
34,45,POINT (-90.53200 29.05020),5/17/2010,C1,0.28,3.70,10.04,142.27
35,44,POINT (-90.52080 28.98980),5/17/2010,C3,0.60,3.76,11.21,158.56
36,43,POINT (-90.52400 28.95000),5/17/2010,C4,0.77,3.97,10.43,153.37
37,42,POINT (-90.48820 28.91380),5/17/2010,C5,0.76,3.87,9.44,133.37
38,40,POINT (-90.48880 28.86820),5/17/2010,C6C,1.05,4.91,7.28,106.16
...,...,...,...,...,...,...,...,...
206,213,POINT (-90.49070 28.86870),10/28/2010,C6C,0.18,4.74,6.75,99.38
207,213-2,POINT (-90.49180 28.86550),10/28/2010,C6C-2,0.05,4.76,6.72,99.08
208,212,POINT (-90.39150 28.83180),10/28/2010,C7,0.17,4.76,6.74,99.31
209,211,POINT (-90.27700 28.78870),10/28/2010,C8,0.07,4.71,6.69,97.88


In [19]:
# Change date format to make it compatible with that of the oil plume trajectory
for i in range(len(data)):
    data.iloc[i, 2] = datetime.strptime(data.iloc[i, 2].replace('/', '-'), '%m-%d-%Y').date()
    data.iloc[i, 2] = data.iloc[i, 2].strftime('%Y-%m-%d')

In [20]:
# Create a shapefile with WGS 1984 coordinate system
data.to_file('Stations.shp', driver='ESRI Shapefile')
sr = arcpy.SpatialReference(4326)
arcpy.management.DefineProjection("Stations.shp", sr)

<Result 'Stations.shp'>

# Spatial Interpolation

### Dissolved Oxygen (mg/L)

In [21]:
# Interpolation and one-value cross validation

arcpy.ga.IDW("Stations.shp", "O2 (mg/L)", "IDW_oxygen", "IDW_oxygen.tif", 0.00488239999999999, 2, "NBRTYPE=Standard S_MAJOR=1.29618154014012 S_MINOR=1.29618154014012 ANGLE=0 NBR_MAX=15 NBR_MIN=10 SECTOR_TYPE=ONE_SECTOR", None)
oxygen_cv_IDW = arcpy.ga.CrossValidation("IDW_oxygen")
print(f"Oxygen IDW RMSE = {str(oxygen_cv_IDW.rootMeanSquare)}")

arcpy.ga.GlobalPolynomialInterpolation("Stations.shp", "O2 (mg/L)", "GPI_oxygen", "GPI_oxygen.tif", 0.00488239999999999, 2, None)
oxygen_cv_GPI = arcpy.ga.CrossValidation("GPI_oxygen")
print(f"Oxygen GPI RMSE = {str(oxygen_cv_GPI.rootMeanSquare)}")

arcpy.ga.LocalPolynomialInterpolation("Stations.shp", "O2 (mg/L)", "LPI_oxygen", "LPI_oxygen.tif", 0.00488239999999999, 0, "NBRTYPE=Standard S_MAJOR=1.29618154014012 S_MINOR=1.29618154014012 ANGLE=0 NBR_MAX=15 NBR_MIN=10 SECTOR_TYPE=ONE_SECTOR", "EXPONENTIAL", None, "NO_USE_CONDITION_NUMBER", None, None, "PREDICTION")
oxygen_cv_LPI = arcpy.ga.CrossValidation("LPI_oxygen")
print(f"Oxygen LPI RMSE = {str(oxygen_cv_LPI.rootMeanSquare)}")

arcpy.ga.RadialBasisFunctions("Stations.shp", "O2 (mg/L)", "RBF_oxygen", "RBF_oxygen.tif", 0.00488239999999999, "NBRTYPE=Standard S_MAJOR=1.29618154014012 S_MINOR=1.29618154014012 ANGLE=0 NBR_MAX=15 NBR_MIN=10 SECTOR_TYPE=ONE_SECTOR", "COMPLETELY_REGULARIZED_SPLINE", None)
oxygen_cv_RBF = arcpy.ga.CrossValidation("RBF_oxygen")
print(f"Oxygen RBF RMSE = {str(oxygen_cv_RBF.rootMeanSquare)}")

Oxygen IDW RMSE = 1.2562260210818204
Oxygen GPI RMSE = 1.1248656411695528
Oxygen LPI RMSE = 1.1142718605064674
Oxygen RBF RMSE = 1.1916096058155428


In [22]:
# Keep interpolations only over water

arcpy.CreateFolder_management(wksp, "oxygen_interpolations")

oxygen_interpolation_tifs = ["IDW_oxygen.tif", "GPI_oxygen.tif", "LPI_oxygen.tif", "RBF_oxygen.tif"]

for tif in oxygen_interpolation_tifs:
    name = "oxygen_interpolations\\" + tif
    out_raster = arcpy.sa.ExtractByMask(tif, "border_Project", "OUTSIDE", '-94.5161412 28.4832588 -89.472622 29.7087412 GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]'); out_raster.save(name)

### Oxygen saturation (%)

In [23]:
# Interpolation and one-value cross validation

arcpy.ga.IDW("Stations.shp", "O2 (% sat)", "IDW_oxygen_sat", "IDW_oxygen_sat.tif", 0.00488239999999999, 2, "NBRTYPE=Standard S_MAJOR=1.29618154014012 S_MINOR=1.29618154014012 ANGLE=0 NBR_MAX=15 NBR_MIN=10 SECTOR_TYPE=ONE_SECTOR", None)
oxygen_sat_cv_IDW = arcpy.ga.CrossValidation("IDW_oxygen_sat")
print(f"Oxygen saturation IDW RMSE = {str(oxygen_sat_cv_IDW.rootMeanSquare)}")

arcpy.ga.GlobalPolynomialInterpolation("Stations.shp", "O2 (% sat)", "GPI_oxygen_sat", "GPI_oxygen_sat.tif", 0.00488239999999999, 2, None)
oxygen_sat_cv_GPI = arcpy.ga.CrossValidation("GPI_oxygen_sat")
print(f"Oxygen saturation GPI RMSE = {str(oxygen_sat_cv_GPI.rootMeanSquare)}")

arcpy.ga.LocalPolynomialInterpolation("Stations.shp", "O2 (% sat)", "LPI_oxygen_sat", "LPI_oxygen_sat.tif", 0.00488239999999999, 0, "NBRTYPE=Standard S_MAJOR=1.29618154014012 S_MINOR=1.29618154014012 ANGLE=0 NBR_MAX=15 NBR_MIN=10 SECTOR_TYPE=ONE_SECTOR", "EXPONENTIAL", None, "NO_USE_CONDITION_NUMBER", None, None, "PREDICTION")
oxygen_sat_cv_LPI = arcpy.ga.CrossValidation("LPI_oxygen_sat")
print(f"Oxygen saturation LPI RMSE = {str(oxygen_sat_cv_LPI.rootMeanSquare)}")

arcpy.ga.RadialBasisFunctions("Stations.shp", "O2 (% sat)", "RBF_oxygen_sat", "RBF_oxygen_sat.tif", 0.00488239999999999, "NBRTYPE=Standard S_MAJOR=1.29618154014012 S_MINOR=1.29618154014012 ANGLE=0 NBR_MAX=15 NBR_MIN=10 SECTOR_TYPE=ONE_SECTOR", "COMPLETELY_REGULARIZED_SPLINE", None)
oxygen_sat_cv_RBF = arcpy.ga.CrossValidation("RBF_oxygen_sat")
print(f"Oxygen saturation RBF RMSE = {str(oxygen_sat_cv_RBF.rootMeanSquare)}")

Oxygen saturation IDW RMSE = 18.167580258042758
Oxygen saturation GPI RMSE = 16.513796643074027
Oxygen saturation LPI RMSE = 16.320188154096286
Oxygen saturation RBF RMSE = 17.28459081642893


In [24]:
# Keep interpolations only over water

arcpy.CreateFolder_management(wksp, "oxygen_sat_interpolations")

oxygen_sat_interpolation_tifs = ["IDW_oxygen_sat.tif", "GPI_oxygen_sat.tif", "LPI_oxygen_sat.tif", "RBF_oxygen_sat.tif"]

for tif in oxygen_sat_interpolation_tifs:
    name = "oxygen_sat_interpolations\\" + tif
    out_raster = arcpy.sa.ExtractByMask(tif, "border_Project", "OUTSIDE", '-94.5161412 28.4832588 -89.472622 29.7087412 GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]'); out_raster.save(name)

### Conductivity (S/m)

In [25]:
# Interpolation and one-value cross validation

arcpy.ga.IDW("Stations.shp", "Cond (S/m)", "IDW_cond", "IDW_cond.tif", 0.00488239999999999, 2, "NBRTYPE=Standard S_MAJOR=1.29618154014012 S_MINOR=1.29618154014012 ANGLE=0 NBR_MAX=15 NBR_MIN=10 SECTOR_TYPE=ONE_SECTOR", None)
cond_cv_IDW_cond = arcpy.ga.CrossValidation("IDW_cond")
print(f"Conductivity IDW RMSE = {str(cond_cv_IDW_cond.rootMeanSquare)}")

arcpy.ga.GlobalPolynomialInterpolation("Stations.shp", "Cond (S/m)", "GPI_cond", "GPI_cond.tif", 0.00488239999999999, 2, None)
cond_cv_GPI_cond = arcpy.ga.CrossValidation("GPI_cond")
print(f"Conductivity GPI RMSE = {str(cond_cv_GPI_cond.rootMeanSquare)}")

arcpy.ga.LocalPolynomialInterpolation("Stations.shp", "Cond (S/m)", "LPI_cond", "LPI_cond.tif", 0.00488239999999999, 0, "NBRTYPE=Standard S_MAJOR=1.29618154014012 S_MINOR=1.29618154014012 ANGLE=0 NBR_MAX=15 NBR_MIN=10 SECTOR_TYPE=ONE_SECTOR", "EXPONENTIAL", None, "NO_USE_CONDITION_NUMBER", None, None, "PREDICTION")
cond_cv_LPI_cond = arcpy.ga.CrossValidation("LPI_cond")
print(f"Conductivity LPI RMSE = {str(cond_cv_LPI_cond.rootMeanSquare)}")

arcpy.ga.RadialBasisFunctions("Stations.shp", "Cond (S/m)", "RBF_cond", "RBF_cond.tif", 0.00488239999999999, "NBRTYPE=Standard S_MAJOR=1.29618154014012 S_MINOR=1.29618154014012 ANGLE=0 NBR_MAX=15 NBR_MIN=10 SECTOR_TYPE=ONE_SECTOR", "COMPLETELY_REGULARIZED_SPLINE", None)
cond_cv_RBF_cond = arcpy.ga.CrossValidation("RBF_cond")
print(f"Conductivity RBF RMSE = {str(cond_cv_RBF_cond.rootMeanSquare)}")

Conductivity IDW RMSE = 0.5120483304703259
Conductivity GPI RMSE = 0.48360936876178534
Conductivity LPI RMSE = 0.4766484648172648
Conductivity RBF RMSE = 0.4899470304993439


In [26]:
# Keep interpolations only over water

arcpy.CreateFolder_management(wksp, "conductivity_interpolations")

cond_tifs = ["IDW_cond.tif", "GPI_cond.tif", "LPI_cond.tif", "RBF_cond.tif"]
for tif in cond_tifs:
    name = "conductivity_interpolations\\" + tif
    out_raster = arcpy.sa.ExtractByMask(tif, "border_Project", "OUTSIDE", '-94.5161412 28.4832588 -89.472622 29.7087412 GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]'); out_raster.save(name)

# Intersection

In [27]:
# Spatiotemporal intersection

arcpy.analysis.Intersect(["oil_spill.shp", "Stations.shp"], "oil_spill_Intersect.shp", "ALL", None, "INPUT")

intersect = gpd.read_file('oil_spill_intersect.shp')

no_matches = list()
for row in range(len(intersect)):
    if intersect.iloc[row, 4] != intersect.iloc[row, 9]:
        no_matches.append(row)
        
intersect = intersect.drop(no_matches)
print(intersect.Date_.unique())
intersect

['2010-07-31' '2010-07-30']


,FID_oil_sp,OBJECTID,Forecast,Shape_Leng,Date_,Shape_Le_1,Shape_Area,FID_Statio,Station_ID,Date,Station,Depth__m_,Cond__S_m_,O2__mg_L_,O2____sat_,geometry
32,273,695,FORECASTLIGHT,772278.643440,2010-07-31,7.514385,0.361957,120,146,2010-07-31,C9,0.06,5.24,6.82,109.45,POINT (-90.22020 28.76630)
71,273,695,FORECASTLIGHT,772278.643440,2010-07-31,7.514385,0.361957,119,145,2010-07-31,C8,0.56,4.82,7.18,113.75,POINT (-90.27630 28.78830)
153,273,695,FORECASTLIGHT,772278.643440,2010-07-31,7.514385,0.361957,118,148,2010-07-31,B9,0.21,5.08,6.41,101.94,POINT (-89.99820 28.83420)
214,272,692,FORECASTLIGHT,357169.847129,2010-07-30,3.491548,0.122079,89,142,2010-07-30,C6C,0.06,4.22,7.80,121.69,POINT (-90.49220 28.86750)
362,273,695,FORECASTLIGHT,772278.643440,2010-07-31,7.514385,0.361957,117,147,2010-07-31,B8,0.03,4.59,7.89,123.98,POINT (-90.03100 28.92450)
472,273,695,FORECASTLIGHT,772278.643440,2010-07-31,7.514385,0.361957,105,160,2010-07-31,A'4,0.01,4.07,9.92,153.70,POINT (-89.56730 28.98350)
579,273,695,FORECASTLIGHT,772278.643440,2010-07-31,7.514385,0.361957,116,149,2010-07-31,B6,0.50,4.46,7.21,112.65,POINT (-90.07670 28.99300)
726,273,695,FORECASTLIGHT,772278.643440,2010-07-31,7.514385,0.361957,110,157,2010-07-31,A5,0.03,4.44,6.97,108.16,POINT (-89.74980 29.07070)
764,273,695,FORECASTLIGHT,772278.643440,2010-07-31,7.514385,0.361957,109,156,2010-07-31,A4,0.07,4.35,7.36,113.89,POINT (-89.75000 29.13420)
